In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
def create_path(relative_path):
  return f'/content/drive/My Drive/MLofi/{relative_path}'

In [60]:
import os
from music21 import *
import numpy as np

In [61]:
# Dataset used : 
# https://www.kaggle.com/zakarii/lofi-hip-hop-midi/metadata

rootdir = "datasets/lofi-hip-hop-midi"

# Holds all the notes of all of our songs!
allSongs = []

for subdirs, dirs, files in os.walk(create_path(rootdir)):
    for file in files:
        currentSongPath = os.path.join(subdirs, file)
        currentSongParse = converter.parse(currentSongPath)
        currentSong = []
        
        for el in currentSongParse.recurse():   
            if isinstance(el, note.Note):
                currentSong.append(str(el.pitch))
            elif isinstance(el, chord.Chord):
                # try sorting it => sorting it doesnt make a change!
                #currCord = [str(n) for n in el.pitches]
                #currentSong.append('.'.join(sorted(currCord)))
                #currentSong.append('.'.join(str(n) for n in el.pitches))
                currentSong.append('.'.join(str(n) for n in el.normalOrder))
            # For now, all of our rests will be of the duration 1 / 8 
            # in the second part of the project, we will take their actual duration!
            elif type(el) == note.Rest:
                # el.duration.quarterLength = 0.5
                currentSong.append(str(el.fullName))
                
        allSongs.append(currentSong)
        

In [62]:
# Padding element is supposed to signify an end to one song, and the beginning of the other
paddingElement = note.Rest()
paddingElement.duration.quarterLength = 16.0

# Currently we are adding a padding element inbetween 2 songs 
# (might work on this in the future, if it gives bad results!)
def flattenSongs(allSongs):
    allNotes = []
    
    for song in allSongs:
        allNotes.append(str(paddingElement.fullName))
        for note in song:
            allNotes.append(note)
        
    return allNotes 
    #maxLenOfSong = max(map(lambda l : len(l), allSongs))
    
    
allNotes = flattenSongs(allSongs)
# print(allNotes)

In [63]:
possibleValues = set(allNotes)
mappingValuesToInt = dict()
mappingIntToValues = dict()
# LTSM works better with int values than with strings, so we need to map out notes (portrayed as strings) to ints
# (we also made a mapIntToVal, so once we need some values with our model, we can return them to their actual representation)
for i, val in enumerate(possibleValues):
    mappingValuesToInt[val] = i
    mappingIntToValues[i] = val

# kako cemo kasnije kad imamo string, pretvoriti u note?

In [64]:
# we will have a list of 20 values for an input, and the output will be a single value
# our model needs to "figure out" what the next note will be using the previous 20 notes!
seqLen = 50

# TODO: FUTURE IDEAS
# in the future, we might change test out what diffrence does it make when we use a diffrent seqLen
# for instance, seqLen = [1, 5, 10, 20, 100, ...]

# inputs and outputs for our model
X_integer_encoded = []
Y_integer_encoded = []
    
for i in range(seqLen, len(allNotes)):
    
    inputValues = allNotes[i - seqLen : i]
    outputValues = allNotes[i]
    
    X_integer_encoded.append([mappingValuesToInt[note]  for note in inputValues]);
    Y_integer_encoded.append(mappingValuesToInt[outputValues]);

# Why do we need the 1 at the end??
X_integer_encoded = np.reshape(X_integer_encoded, (len(X_integer_encoded), seqLen, 1))
# need to add normalization for X???

In [65]:
import tensorflow as tf
from tensorflow import keras

In [66]:
X = keras.utils.to_categorical(X_integer_encoded)
Y = keras.utils.to_categorical(Y_integer_encoded)

In [67]:
X.shape

(2779, 50, 342)

In [68]:
Y.shape

(2779, 342)

In [69]:
## HIPOTEZA #1: MODEL JE PREVIŠE JEDNOSTAVAN DA BI MOGAO DA VRŠI DOBRU PREDIKCIJU
## HIPOTEZA #2: SEQUENCE LENGTH 20 JE PREVIŠE KRATAK DA BI SE MODELOVALI KOMPLEKSNI ODNOSI U MUZICI
# Making a very basic model to test the pipeline
# model = keras.models.Sequential()
# model.add(keras.layers.Input(shape=(X.shape[1], X.shape[2])))
# model.add(keras.layers.LSTM(256))
# model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.Dense(Y.shape[1], activation=keras.activations.softmax))
# model.summary()

In [70]:
# model.compile(loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()], optimizer=keras.optimizers.Adam())

In [71]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(X.shape[1], X.shape[2])))
model.add(keras.layers.LSTM(512, return_sequences=True))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.LSTM(512, return_sequences=True))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.LSTM(512))
model.add(keras.layers.Dense(256))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(Y.shape[1]))
model.add(keras.layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [72]:
# filepath = create_path("weights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5")
# checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

In [73]:
model.fit(X, Y, epochs=60, batch_size=64)

Epoch 1/60
44/44 [==============================] - 15s 113ms/step - loss: 5.4592
Epoch 2/60
44/44 [==============================] - 5s 110ms/step - loss: 5.0547
Epoch 3/60
44/44 [==============================] - 5s 110ms/step - loss: 4.8749
Epoch 4/60
44/44 [==============================] - 5s 110ms/step - loss: 4.6792
Epoch 5/60
44/44 [==============================] - 5s 110ms/step - loss: 4.4782
Epoch 6/60
44/44 [==============================] - 5s 110ms/step - loss: 4.3433
Epoch 7/60
44/44 [==============================] - 5s 110ms/step - loss: 4.1871
Epoch 8/60
44/44 [==============================] - 5s 110ms/step - loss: 4.0296
Epoch 9/60
44/44 [==============================] - 5s 111ms/step - loss: 3.8822
Epoch 10/60
44/44 [==============================] - 5s 110ms/step - loss: 3.7590
Epoch 11/60
44/44 [==============================] - 5s 110ms/step - loss: 3.6442
Epoch 12/60
44/44 [==============================] - 5s 110ms/step - loss: 3.4965
Epoch 13/60
44/44 [=====

In [74]:
noteCount = len(possibleValues)
notes = np.array(list(possibleValues))

r = np.random.randint(low=0, high=noteCount, size=seqLen)
rn = notes[r]
irn = [mappingValuesToInt[x] for x in rn]

In [75]:
pattern = irn
prediction = []
for note_index in range(500):
    rp = np.reshape(np.array(pattern), (1, seqLen, 1))
    cp = keras.utils.to_categorical(rp, num_classes=noteCount)

    p = model.predict(cp, verbose=0)

    # print(p)
    i = np.argmax(p)
    result = mappingIntToValues[i]
    prediction.append(result)

    pattern.append(i)
    pattern = pattern[1:len(pattern)]
prediction

['G#4',
 'G#5',
 'F#4',
 'Imperfect Longa Rest',
 'F#4',
 'D5',
 'Half Tuplet of 6/5ths (1 2/3 QL) Rest',
 'Half Tuplet of 6/5ths (1 2/3 QL) Rest',
 '8.11.1.4',
 '8.11.1.4',
 '6.9.11.2',
 '11.2',
 '1.2.4.6.9',
 '7.11',
 'A4',
 'A4',
 'Whole Rest',
 'Double Dotted Quarter Rest',
 '4.7.9.0',
 'G#4',
 '4.7.9.0',
 'Half Rest',
 'A4',
 'E5',
 'G4',
 '16th Rest',
 'Double Dotted Quarter Rest',
 'D2',
 'Imperfect Longa Rest',
 '5.8.0',
 '7.10.0.3',
 '9',
 'F4',
 'G2',
 '5.7.0',
 'G#4',
 'E-5',
 'G#4',
 'E-5',
 '0.3.5.8',
 '10',
 '9.10.2.5',
 'F5',
 'F5',
 'G5',
 'C5',
 'G#5',
 'C5',
 'E-5',
 'G#4',
 'G#5',
 'D5',
 'F5',
 'G#4',
 'F5',
 'Imperfect Longa Rest',
 '11.2.6',
 '11.2.6',
 'D5',
 'Dotted Quarter Rest',
 'E3',
 'G#2',
 '4.7',
 '7.11.2',
 'B-3',
 'B-3',
 'B-3',
 'D5',
 'F#5',
 'D5',
 'F5',
 'D5',
 'C#5',
 'B4',
 'B-4',
 'B-4',
 'C6',
 'E3',
 'G4',
 'G#5',
 'A4',
 'E-5',
 'B-4',
 'C#5',
 'B4',
 'G#4',
 '1.3',
 '1.3',
 'G#5',
 'C#4',
 'Imperfect Longa Tuplet of 48/25ths (8 1/3 QL) Rest',

In [76]:
import re

In [77]:
ptn_1 = re.compile(r'[A-Z]')
ptn_2 = re.compile(r'Rest')

s = stream.Stream()
for element in prediction:
  if ptn_2.search(element):
    s.append(note.Rest(element))
  elif ptn_1.search(element) is not None:
    s.append(note.Note(element))
  else:
    s.append(chord.Chord([int(x) for x in element.split('.')]))

In [78]:
# s.show('text')
s.write('midi', fp=create_path('output/test/03.mid'))

'/content/drive/My Drive/MLofi/output/test/02.mid'